In [25]:
import os
import requests
import json
import pandas as pd
import yfinance as yf
import pendulum
import matplotlib.pyplot as plt
from MCForecastTools import MCSimulation

from dotenv import load_dotenv
import alpaca_trade_api as tradeapi


%matplotlib inline

In [35]:
load_dotenv('keys.env')

True

In [27]:
import numpy as np
from pandas_datareader import data as wb
import seaborn as sns
from scipy.stats import norm
import datetime as dt
from mc_simulation import MonteCarloSimulation as mcs

In [4]:
#start_date=dt.datetime(2018,1,1)
#end_date=dt.datetime(2020,1,1)
#print(start)

In [5]:
#ticker = 'TSLA'
#data = pd.DataFrame(['TSLA'])
#wb.DataReader("TSLA", 'yahoo', start=start_date, end=end_date)["Close"]
#Plot
#data.plt(figsize=(15,6))

In [6]:
#price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   #interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   #actions=False)
#time_series = list(price_history['Close'])
#dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]

#plt.plot(time_series)
#display(dt_list)
#display(time_series)


In [7]:
#political_portfolio_dict_stock_list = [{'Mark_Warner': ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']}, 
 #{'Greg_Gianforte': ['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'RHT']},
 #{'Nancy_Pelosi': ['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']}]
                 
#Mark Warner: SPDR s&p 500 etf, Akre focus fund, t. rowe price group, coho relative value equity fund, vanguard 

#Greg Gianforte: invesco qqq, SPDr s&p 500, vanguard, FICO, Workiva, red hat inc

#Nancy Pelosi: apple, visa, disney, salesforce, facebook, comcast

In [28]:
warner_weights=[0.24, 0.23, 0.22, 0.22, 0.09]
warner_investment_capital=116000000.00

#Warner range of investments from $80m to $348m. Maximum value of assets assumed in calculations.
#investment capital reduced by 2/3 to allow for easier analysis and accomodate top 1/3 of stock 
#investment portfolio. Net worth assumed as investment capital. AMG GW&K Municipal bond fund removed from 
#top 1/3 of stock assets to simplify calculations.

gianforte_weights=[0.26, 0.22, 0.35, 0.08, 0.05, 0.04]
gianforte_investment_capital = 94500000.00

#weights of stocks making up 1/2 of Gianforte investment portfolio. Different Vanguard funds summed. 
#investment capital reduced by 1/2 to allow for easier analysis and accomodate top
#1/2 of stock investment portfolio. Net worth assumed as investment capital. 

pelosi_weights=[0.40, 0.40, 0.08, 0.08, 0.02, 0.02]
pelosi_investment_capital= 64000000.00
                
#Pelosi range of investments from $54m to $256m. Max value investments/assets assumed in calculations. 
#investment capital reduced by 3/4 to allow for easier analysis and accomodate top
#1/4 of stock investment portfolio. Net worth assumed as investment capital.

In [29]:
warner_tickers= ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']
gianforte_tickers=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM']
pelosi_tickers=['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']

#timeframe='1Day'

#start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
#end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [10]:
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_df.columns = pd.MultiIndex.from_product([warner_df.columns, ['Close']])
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)
#display(warner_close_data)

In [33]:
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()
timeframe="1Day"
load_dotenv('keys.env')

# Set Alpaca API key and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)
print(warner_tickers)
warner_prices_df = alpaca.get_bars(warner_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)

#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
spy_df = warner_prices_df[warner_prices_df['symbol']=='SPY'].drop('symbol',axis=1)
akrex_df = warner_prices_df[warner_prices_df['symbol']=='AKREX'].drop('symbol',axis=1)
trow_df = warner_prices_df[warner_prices_df['symbol']=='TROW'].drop('symbol',axis=1)
cohox_df = warner_prices_df[warner_prices_df['symbol']=='COHOX'].drop('symbol',axis=1)
vti_df = warner_prices_df[warner_prices_df['symbol']=='VTI'].drop('symbol',axis=1)
warner_prices_df = pd.concat([spy_df, akrex_df, trow_df, cohox_df, vti_df],axis=1,keys=['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI'])
display(warner_prices_df)
#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
warner_MC_30yr = MCSimulation(
    portfolio_data = warner_prices_df,
    weights = warner_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
#MC_30yr.portfolio_data.head()

['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']


SPY                                        \
                             open      high      low   close     volume   
timestamp                                                                 
2017-05-01 04:00:00+00:00  238.68  239.1700  238.195  238.63   66882521   
2017-05-02 04:00:00+00:00  238.84  238.9800  238.300  238.76   57375732   
2017-05-03 04:00:00+00:00  238.29  238.8775  237.700  238.52   73137731   
2017-05-04 04:00:00+00:00  238.83  238.9225  237.780  238.74   61462732   
2017-05-05 04:00:00+00:00  239.19  239.7200  238.680  239.68   62001269   
...                           ...       ...      ...     ...        ...   
2020-04-27 04:00:00+00:00  285.12  288.2700  284.620  287.10   78438608   
2020-04-28 04:00:00+00:00  291.02  291.4000  285.400  285.69  106308551   
2020-04-29 04:00:00+00:00  291.54  294.8800  290.410  293.15  118756595   
2020-04-30 04:00:00+00:00  291.71  293.3239  288.590  290.63  123003721   
2020-05-01 04:00:00+00:00  285.31  290.6572  281.520  282.83  125180028   

                                                  AKREX           ...  COHOX  \
                          trade_count        vwap  open high low  ... volume   
timestamp                                                         ...          
2017-05-01 04:00:00+00:00      180740  238.682276   NaN  NaN NaN  ...    NaN   
2017-05-02 04:00:00+00:00      176648  238.630973   NaN  NaN NaN  ...    NaN   
2017-05-03 04:00:00+00:00      223568  238.418100   NaN  NaN NaN  ...    NaN   
2017-05-04 04:00:00+00:00      208872  238.541805   NaN  NaN NaN  ...    NaN   
2017-05-05 04:00:00+00:00      204885  239.253410   NaN  NaN NaN  ...    NaN   
...                               ...         ...   ...  ...  ..  ...    ...   
2020-04-27 04:00:00+00:00      430206  286.528977   NaN  NaN NaN  ...    NaN   
2020-04-28 04:00:00+00:00      582616  287.789471   NaN  NaN NaN  ...    NaN   
2020-04-29 04:00:00+00:00      643390  292.698771   NaN  NaN NaN  ...    NaN   
2020-04-30 04:00:00+00:00      650783  290.584103   NaN  NaN NaN  ...    NaN   
2020-05-01 04:00:00+00:00      639987  284.092832   NaN  NaN NaN  ...    NaN   

                                               VTI                      \
                          trade_count vwap    open      high       low   
timestamp                                                                
2017-05-01 04:00:00+00:00         NaN  NaN  122.93  123.1942  122.6200   
2017-05-02 04:00:00+00:00         NaN  NaN  123.09  123.2500  122.7000   
2017-05-03 04:00:00+00:00         NaN  NaN  122.59  122.7800  122.2800   
2017-05-04 04:00:00+00:00         NaN  NaN  122.83  122.8300  122.1601   
2017-05-05 04:00:00+00:00         NaN  NaN  122.95  123.3300  122.6800   
...                               ...  ...     ...       ...       ...   
2020-04-27 04:00:00+00:00         NaN  NaN  142.65  144.6600  141.0000   
2020-04-28 04:00:00+00:00         NaN  NaN  146.17  146.4800  143.3700   
2020-04-29 04:00:00+00:00         NaN  NaN  146.81  148.5400  146.0300   
2020-04-30 04:00:00+00:00         NaN  NaN  146.78  147.0500  145.1200   
2020-05-01 04:00:00+00:00         NaN  NaN  143.12  143.6000  139.8000   

                                                                    
                            close   volume trade_count        vwap  
timestamp                                                           
2017-05-01 04:00:00+00:00  122.93  1983507       13416  122.905242  
2017-05-02 04:00:00+00:00  122.93  1756917       13221  122.864796  
2017-05-03 04:00:00+00:00  122.63  3690976       21693  122.576499  
2017-05-04 04:00:00+00:00  122.72  1564459       21780  122.588963  
2017-05-05 04:00:00+00:00  123.33  2057663       12588  122.930443  
...                           ...      ...         ...         ...  
2020-04-27 04:00:00+00:00  144.14  4968198       39127  143.765100  
2020-04-28 04:00:00+00:00  143.69  3594939       31654  144.697177  
2020-04-29 04:00:00+00:00  147.76  4990906       37806  147.63

KeyError: 'Close'

In [26]:
warner_MC_30yr.calc_cumulative_return()

NameError: name 'warner_MC_30yr' is not defined

In [ ]:
warner_MC_sim_line_plot = warner_MC_30yr.plot_simulation()

In [ ]:
warner_MC_summary_stats = warner_MC_30yr.summarize_cumulative_return()
print(warner_MC_summary_stats)

In [ ]:
#warner_returns=warner_close_data.pct_change()
#warner_returns.head()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()

#display(warner_returns.dropna())

In [ ]:
#warner_portfolio_returns=warner_returns.dot(warner_weights)
#warner_portfolio_returns.head()

In [ ]:
#warner_variance=warner_returns.cov()*252
#warner_variance

In [ ]:
#warner_portfolio_variance=np.transpose(warner_weights)@warner_variance@warner_weights
#warner_portfolio_volitility=np.sqrt(warner_portfolio_variance)
#display(f"warner_portfolio_variance is", warner_portfolio_variance)
#display(f"warner_portfolio_volitility is", warner_portfolio_volitility)

In [ ]:
#gianforte_stock_info=yf.Tickers(gianforte_tickers)
#gianforte_df=gianforte_stock_info.history(period='5y')
#gianforte_df=pd.DataFrame(gianforte_df)
#gianforte_close_data=gianforte_df['Close']
#gianforte_close_data=gianforte_close_data.reindex(columns= gianforte_tickers)
#display(gianforte_close_data)

In [ ]:
print(gianforte_tickers)
gianforte_prices_df = alpaca.get_bars(gianforte_tickers,timeframe,start_date,end_date).df

In [ ]:
print(gianforte_tickers)
gianforte_prices_df = alpaca.get_bars(gianforte_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
qqq_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
spy_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
vti_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
fico_df =gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
wk_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
crm_df = gianforte_prices_df[gianforte_prices_df['symbol']=='QQQ'].drop('symbol',axis=1)
gianforte_prices_df = pd.concat([qqq_df,spy_df,vti_df,fico_df,wk_df,crm_df],axis=1,keys=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM'])


#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
gianforte_MC_30yr = MCSimulation(
    portfolio_data = gianforte_prices_df,
    weights = gianforte_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
gianforte_MC_30yr.portfolio_data.head()

In [ ]:
gianforte_MC_30yr.calc_cumulative_return()

In [ ]:
gianforte_MC_summary_stats = gianforte_MC_30yr.summarize_cumulative_return()
print(gianforte_MC_summary_stats)

In [ ]:
gianforte_MC_30yr.calc_cumulative_return()

In [ ]:
gianforte_MC_sim_line_plot = gianfote_MC_30yr.plot_simulation()

In [ ]:
gianforte_returns=gianforte_close_data.pct_change()
mean_gianforte_returns=gianforte_returns.mean()
std_gianforte_returns=gianforte_returns.std()

In [ ]:
#pelosi_stock_info=yf.Tickers(pelosi_tickers)
#pelosi_df=pelosi_stock_info.history(period='5y')
#pelosi_df=pd.DataFrame(pelosi_df)
#pelosi_close_data=pelosi_df['Close']
#pelosi_close_data=pelosi_close_data.reindex(columns= pelosi_tickers)
#display(pelosi_close_data)

In [ ]:
print(pelosi_tickers)
pelosi_prices_df = alpaca.get_bars(pelosi_tickers,timeframe,start_date,end_date).df
#warner_stock_info=yf.Tickers(warner_tickers)
#warner_df=warner_stock_info.history(period='5y')
#warner_df=pd.DataFrame(warner_df)
#warner_close_data=warner_df['Close']
#warner_close_data=warner_close_data.reindex(columns=warner_tickers)  
aapl_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
v_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
dis_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
crm_df =pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
meta_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
cmcsa_df = pelosi_prices_df[pelosi_prices_df['symbol']=='AAPL'].drop('symbol',axis=1)
pelosi_prices_df = pd.concat([aapl_df,v_df,dis_df,crm_df,meta_df,cmcsa_df],axis=1,keys=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM'])


#warner_returns=warner_close_data.pct_change()
#mean_warner_returns=warner_returns.mean()
#std_warner_returns=warner_returns.std()
pelosi_MC_30yr = MCSimulation(
    portfolio_data = pelosi_prices_df,
    weights = pelosi_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
pelosi_MC_30yr.portfolio_data.head()

In [ ]:
pelosi_MC_sim_line_plot = pelosi_MC_30yr.plot_simulation()

In [ ]:
pelosi_MC_summary_stats = pelosi_MC_30yr.summarize_cumulative_return()
print(pelosi_MC_summary_stats)

In [ ]:
pelosi_returns=pelosi_close_data.pct_change()
mean_pelosi_returns=pelosi_returns.mean()
std_pelosi_returns=pelosi_returns.std()

In [ ]:
spy_shares_warner= 110476
akrex_shares_warner= 762286
trow_shares_warner= 220000
cohox_shares_warner= 1772222
vti_shares_warner= 74359

qqq_shares_gianforte= 149271
spy_shares_gianforte= 82500
vti_shares_gianforte= 235577
fico_shares_gianforte= 39171
wk_shares_gianforte= 181730
rht_shares_gianforte= 25116

aapl_shares_pelosi=548180
visa_shares_pelosi=193939
dis_shares_pelosi=46694
crm_shares_pelosi=37372
meta_shares_pelosi=7442
cmcsa_shares_pelosi=35774

#total shares calculated based on 2018 median share prices for each stock in portfolio b/c most recent portfolio data
#from 2018. should 

In [ ]:
stock_investment= pelosi_investment_capital * 0.02
#spy 2018 median share price =252
#akrex 2018 median share price = 35
#trow 2018 median share price = 116
#cohox 2018 median share price = 14.4
#vti 2018 median share price = 140.4

#qqq 2018 med share price = 164.6
#fico 2018 med share price = 193
#wk 2018 med share price = 26
#rht 2018 med share price = 150.5

#aapl 2018 med share price = 46.7
#visa 2018 med share price = 132
#disney 2018 med share price = 109.65
#crm 2018 med share price = 137
#meta 2018 med share price = 172
#cmcsa 2018 med share price = 35.78

In [ ]:
stock_investment/35.78